<a href="https://colab.research.google.com/github/StevenHSKim/tensorflow_study/blob/main/pj3_%EA%B0%9C_%EA%B3%A0%EC%96%91%EC%9D%B4_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dog vs Cat Classification

이전까진 미리 전처리된 데이터를 사용해 딥러닝을 돌려보았습니다.

하지만 실전에선 누가 전처리해주지 않고 우리가 직접하는 경우가 많겠죠.

쌩 이미지 데이터 백만개를 주었을 때 딥러닝을 어떻게 돌리는지 알아보기위해

이번엔 개/고양이 사진을 구분하는 AI를 만들어보도록 합시다.

Kaggle 데이터 다운로드 받는 방법부터 시작

###Kaggle에서 데이터셋 사용하는 방법
1. os로 다운로드 받은 후, !kaggle로 코랩 content에 저장
2. !unzip으로 압축 풀기
3. 이미지 전처리 작업에서 cat,dog 폴더 직접 만들어주기(/dataset/ 폴더 안에 /cat/과 /dog/ 만들기)

In [ ]:
# 코랩에서 kaggle dataset 다운로드
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats

 98% 793M/812M [00:08<00:00, 173MB/s]
100% 812M/812M [00:08<00:00, 104MB/s]


In [ ]:
# 코랩에서 압축 풀기
!unzip -q dogs-vs-cats.zip -d .

In [ ]:
!unzip -q train.zip -d .

In [ ]:
# 이미지 파일 개수 세기
import os
print(len(os.listdir('/content/train/')))

25000


이번엔 이미지를 숫자로 변환해볼텐데 직접 opencv 라이브러리를 이용해서 손코딩하거나
tf keras를 이용해서 쉽게 하거나 둘 중 하나만 하면 됩니다.

여기선 후자로 진행합니다.

폴더 내의 이미지들을 바로 데이터셋 자료로 만들어주는 고마운 함수가 있습니다.



일단 그 함수를 쓰기 위해선 이미지들을 폴더별로 분류하는 작업이 필요합니다.

개 사진은 dog 폴더로, 고양이 사진은 cat 폴더로 전부 밀어넣어야하는데 사진이 2만5천장이라

손수하려면 드래그하는데도 힘들겠네요. 파이썬 코드로 이미지를 옮겨보도록 합시다.


이미지를 각각 폴더에 분류를 잘 했다면

이후 전처리를 요약하자면 이게 끝입니다.


# 데이터셋 = tf.keras.preprocessing.image_dataset_from_directory(
#    '사진들어있는폴더경로',
#    image_size=(64,64)
# )


근데 나중에 혼자 공부하며 주의점은 사진 이름이 1.jpg 2.jpg 이렇게 숫자로만 되어있으면 에러가 날 수 있어서

숫자로만 되어있으면 앞에 cat1.jpg cat2.jpg 이렇게 이름을 변경해주셔야합니다.

os.rename() 을 가져다 쓰시면 파일명 변경해줍니다.

#이미지 전처리

In [ ]:
import tensorflow as tf
import shutil

# tf.keras.preprocessing.image_dataset_from_directory() 이용하기 위한 사전 작업 - cat/dog 폴더 구분
# 먼저, content에 dataset 폴더 만들고 cat, dog 폴더 만들기
for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
  if 'dog' in i:
    shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)

In [ ]:
# tf.keras 이용해서 이미지를 숫자화
# train_ds에는 ( x(이미지를 행렬로 바꾼 데이터), y(개,고양이 정답 0 또는 1) )가 담김
# train_ds를 모델에 집어넣으면 학습 끝
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=32,

    # validation data도 준비
    subset='training',
    validation_split=0.2, # 20% 만큼 validation dataset으로 쪼개기
    seed=1234
)

# validation dataset 만들 때 이 형식 따라야 함.
# 위 train_ds은 'training'으로 이름짓고, 아래 val_ds는 'validation'으로 이름 짓기
# train_ds는 전체 데이터 중 80%, val_ds는 전체 데이터 중 20%
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/dataset/',
    image_size=(64,64),
    batch_size=32,

    subset='validation',
    validation_split=0.2,
    seed=1234
)

print(train_ds)
# print 해보면, 위에서 언급한 대로
# ((shape=(None, 64, 64, 3), (shape=(None,)) 모양으로 나오는 것을 알 수 있음.


#------------------성능 높히는 코드(보통 사용함)------------------
# 이미지 데이터는 현재 텐서 형태로 0~255의 값을 가지고 있음. 이를 0~1로 압축하기
def preprocessing_function(i, 정답):
  i = tf.cast(i/255.0, tf.float32) # 텐서를 다루기 때문에 i = i/255.0 불가 -> tf.cast() 사용
  return i, 정답

# map(func): 데이터에 전부 func를 적용해라
train_ds = train_ds.map(preprocessing_function)
val_ds = val_ds.map(preprocessing_function)
#-----------------------------------------------------------

# 0~1로 압축한 모습 확인
# for i, 정답 in train_ds.take(1): # take(1): 1개만 뽑아서 확인
#   print(i)
#   print(정답)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


#학습모델 만들기


이전에 했던 것과 유사합니다.
이번엔 Dropout Layer를 추가해볼건데 이게 뭐냐면 윗 레이어에 있는 노드들의 일부를 버려주는 레이어입니다.

### tf.keras.layers.Dropout(0.2)

이렇게 쓰면 윗 레이어의 노드의 20%를 버려줍니다. (0으로 만들어줍니다)

이 레이어는 트레이닝용 데이터를 외워버려 validation 데이터보다 매우 정확도가 높아지는 overfitting 현상을 완화하기 위해서 사용합니다.

In [ ]:
# 모델 만들기
model = tf.keras.Sequential([
    # Convolution 3번 반복 해보기
    tf.keras.layers.Conv2D(32, (3,3) ,padding="same", activation='relu', input_shape=(64,64,3)), # RGB 채널: 3
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(64, (3,3) ,padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    # dropout: overfitting 방지 위해 노드가 많아보이면 줄임 (여기선 20%)
    tf.keras.layers.Dropout(0.2), # 위치는 아무데나. 보통 Conv에선 Pooling 끝나고.

    tf.keras.layers.Conv2D(128, (3,3) ,padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"), # binary_crossentropy 문제는 sigmoid로 - (분류 문제) 0~1 사이의 확률 제시
])

# 모델 Summary
model.summary()

# 모델 컴파일 및 실행
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])
model.fit(train_ds, validation_data=(val_ds), epochs=5)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 64)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 64)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 128)      